In [1]:
from datasets import load_dataset
from nltk.tokenize import wordpunct_tokenize
import dask.dataframe as dd
import pandas as pd
dataset = load_dataset('ag_news')
import tracemalloc
import gc

/home/t/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf

print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib

# List all devices
print(device_lib.list_local_devices())

2024-09-28 18:55:36.143940: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 18:55:36.836883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 18:55:37.664797: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 18:55:37.961377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-28 18:55:38.419670: I tensorflow/core/platform/cpu_feature_guar

GPUs Available:  2
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2240182411500555318
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9979297792
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4585022446947356386
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 5934940160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10708231544892424385
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:05:00.0, compute capability: 8.6"
xla_global_id: 2144165316
]


I0000 00:00:1727538947.878417   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727538947.878784   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727538947.878837   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727538947.878853   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727538947.878867   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
text=[]
label=[]
for row in dataset['train']['text']+dataset['test']['text']:
    text.append(wordpunct_tokenize(row.lower()))
for row in dataset['train']['label']+dataset['test']['label']:
    label.append(row)

In [4]:
word_dict={'PADDING':0}
for sent in text:    
    for token in sent:        
        if token not in word_dict:
            word_dict[token]=len(word_dict)

In [5]:
MAX_SENT_LENGTH=256

news_words = []
for sent in text:       
    sample=[]
    for token in sent:     
        sample.append(word_dict[token])
    sample = sample[:MAX_SENT_LENGTH]
    news_words.append(sample+[0]*(MAX_SENT_LENGTH-len(sample)))


In [6]:
import numpy as np
news_words=np.array(news_words,dtype='int32') 
label=np.array(label,dtype='int32') 

In [7]:
index=np.arange(len(label))
train_index=index[:120000]
np.random.shuffle(train_index)
test_index=index[120000:]

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model, load_model
from tensorflow.keras import backend as K
from sklearn.metrics import *
from keras.optimizers import *
import keras.backend as K
import keras

In [9]:
import numpy as np
news_words=np.array(news_words,dtype='int32') 
label=np.array(label,dtype='int32')

In [10]:
import random
index=np.arange(len(label))
train_index=index[:120000]
test_index=index[120000:]

In [11]:

from tensorflow.keras.layers import Input, Embedding, Dropout, Lambda, LayerNormalization, Dense, Activation, Dot, Flatten, add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dropout, Lambda, LayerNormalization, Dense, Activation, Reshape, TimeDistributed, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

class Fastformer(Layer):
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head * size_per_head
        self.now_input_shape = None
        super(Fastformer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.now_input_shape = input_shape
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True) 
        self.Wq = self.add_weight(name='Wq', 
                                  shape=(self.output_dim,self.nb_head),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.Wk = self.add_weight(name='Wk', 
                                  shape=(self.output_dim,self.nb_head),
                                  initializer='glorot_uniform',
                                  trainable=True)
        
        self.WP = self.add_weight(name='WP', 
                                  shape=(self.output_dim,self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        
        super(Fastformer, self).build(input_shape)
        
    def call(self, x):
        if len(x) == 2:
            Q_seq,K_seq = x
        elif len(x) == 4:
            Q_seq,K_seq,Q_mask,K_mask = x #different mask lengths, reserved for cross attention

        Q_seq = K.dot(Q_seq, self.WQ)        
        Q_seq_reshape = K.reshape(Q_seq, (-1, self.now_input_shape[0][1], self.nb_head*self.size_per_head))

        Q_att=  K.permute_dimensions(K.dot(Q_seq_reshape, self.Wq),(0,2,1))/ self.size_per_head**0.5

        if len(x)  == 4:
            Q_att = Q_att-(1-K.expand_dims(Q_mask,axis=1))*1e8

        Q_att = K.softmax(Q_att)
        Q_seq = K.reshape(Q_seq, (-1,self.now_input_shape[0][1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1,self.now_input_shape[1][1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))

        Q_att = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=3),self.size_per_head,axis=3))(Q_att)
        global_q = K.sum(multiply([Q_att, Q_seq]),axis=2)
        
        global_q_repeat = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=2), self.now_input_shape[1][1],axis=2))(global_q)

        QK_interaction = multiply([K_seq, global_q_repeat])
        QK_interaction_reshape = K.reshape(QK_interaction, (-1, self.now_input_shape[0][1], self.nb_head*self.size_per_head))
        K_att = K.permute_dimensions(K.dot(QK_interaction_reshape, self.Wk),(0,2,1))/ self.size_per_head**0.5
        
        if len(x)  == 4:
            K_att = K_att-(1-K.expand_dims(K_mask,axis=1))*1e8
            
        K_att = K.softmax(K_att)

        K_att = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=3),self.size_per_head,axis=3))(K_att)

        global_k = K.sum(multiply([K_att, QK_interaction]),axis=2)
     
        global_k_repeat = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=2), self.now_input_shape[0][1],axis=2))(global_k)
        #Q=V
        QKQ_interaction = multiply([global_k_repeat, Q_seq])
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head*self.size_per_head))
        QKQ_interaction = K.dot(QKQ_interaction, self.WP)
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head,self.size_per_head))
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = QKQ_interaction+Q_seq
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head*self.size_per_head))

        return QKQ_interaction
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)



class Fastformer2(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        self.now_input_shape=None
        super(Fastformer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.now_input_shape=input_shape
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True) 
        self.Wq = self.add_weight(name='Wq', 
                                  shape=(self.output_dim,self.nb_head),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.Wk = self.add_weight(name='Wk', 
                                  shape=(self.output_dim,self.nb_head),
                                  initializer='glorot_uniform',
                                  trainable=True)
        
        self.WP = self.add_weight(name='WP', 
                                  shape=(self.output_dim,self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        
        
        super(Fastformer, self).build(input_shape)
        
    def call(self, x):
        if len(x) == 2:
            Q_seq,K_seq = x
        elif len(x) == 4:
            Q_seq,K_seq,Q_mask,K_mask = x #different mask lengths, reserved for cross attention
        Q_seq = tf.linalg.matmul(Q_seq, self.WQ)        
        Q_seq_reshape = tf.reshape(Q_seq, (-1, self.now_input_shape[0][1], self.nb_head*self.size_per_head))

        Q_att = tf.transpose(tf.linalg.matmul(Q_seq_reshape, self.Wq), perm=(0, 2, 1)) / (self.size_per_head**0.5)
        #Q_seq = K.dot(Q_seq, self.WQ)        
        #Q_seq_reshape = K.reshape(Q_seq, (-1, self.now_input_shape[0][1], self.nb_head*self.size_per_head))

        #Q_att=  K.permute_dimensions(K.dot(Q_seq_reshape, self.Wq),(0,2,1))/ self.size_per_head**0.5

        if len(x)  == 4:
            #Q_att = Q_att-(1-K.expand_dims(Q_mask,axis=1))*1e8
            Q_att = Q_att - (1 - tf.expand_dims(Q_mask, axis=1)) * 1e8
        """
        Q_att = K.softmax(Q_att)
        Q_seq = K.reshape(Q_seq, (-1,self.now_input_shape[0][1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1,self.now_input_shape[1][1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))

        Q_att = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=3),self.size_per_head,axis=3))(Q_att)
        global_q = K.sum(multiply([Q_att, Q_seq]),axis=2)
        
        global_q_repeat = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=2), self.now_input_shape[1][1],axis=2))(global_q)

        QK_interaction = multiply([K_seq, global_q_repeat])
        QK_interaction_reshape = K.reshape(QK_interaction, (-1, self.now_input_shape[0][1], self.nb_head*self.size_per_head))
        K_att = K.permute_dimensions(K.dot(QK_interaction_reshape, self.Wk),(0,2,1))/ self.size_per_head**0.5
        
        if len(x)  == 4:
            K_att = K_att-(1-K.expand_dims(K_mask,axis=1))*1e8
            
        K_att = K.softmax(K_att)

        K_att = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=3),self.size_per_head,axis=3))(K_att)

        global_k = K.sum(multiply([K_att, QK_interaction]),axis=2)
     
        global_k_repeat = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,axis=2), self.now_input_shape[0][1],axis=2))(global_k)
        #Q=V
        QKQ_interaction = multiply([global_k_repeat, Q_seq])
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head*self.size_per_head))
        QKQ_interaction = K.dot(QKQ_interaction, self.WP)
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head,self.size_per_head))
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = QKQ_interaction+Q_seq
        QKQ_interaction = K.permute_dimensions(QKQ_interaction, (0,2,1,3))
        QKQ_interaction = K.reshape(QKQ_interaction, (-1,self.now_input_shape[0][1], self.nb_head*self.size_per_head))
        """
        #many operations can be optimized if higher versions are used. 
        # Softmax
        #Q_att = tf.nn.softmax(Q_att)

        # Reshape and transpose
        #Q_seq = tf.reshape(Q_seq, (-1, self.now_input_shape[0][1], self.nb_head, self.size_per_head))
        #Q_seq = tf.transpose(Q_seq, perm=(0, 2, 1, 3))
        # Ensure WK is shaped as [size_per_head, size_per_head]
        #if self.WK.shape != (self.size_per_head, self.size_per_head):
        #    self.WK = tf.Variable(tf.random.normal([self.size_per_head, self.size_per_head]))
        Q_att = K.softmax(Q_att)
        Q_seq = K.reshape(Q_seq, (-1,self.now_input_shape[0][1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        # Matrix multiplication and reshaping
        #K_seq = tf.matmul(Q_seq, self.WK)
        K_seq = K.dot(K_seq, self.WK)
        
        K_seq = tf.reshape(K_seq, (-1, self.now_input_shape[1][1], self.nb_head, self.size_per_head))
        K_seq = tf.transpose(K_seq, perm=(0, 2, 1, 3))

        # Repeat and sum
        Q_att = tf.repeat(tf.expand_dims(Q_att, axis=3), repeats=self.size_per_head, axis=3)
        global_q = tf.reduce_sum(tf.multiply(Q_att, Q_seq), axis=2)

        # Repeat elements and reshape for interaction
        global_q_repeat = tf.repeat(tf.expand_dims(global_q, axis=2), self.now_input_shape[1][1], axis=2)
        QK_interaction = tf.multiply(K_seq, global_q_repeat)
        QK_interaction_reshape = tf.reshape(QK_interaction, (-1, self.now_input_shape[0][1], self.nb_head * self.size_per_head))

        # Another matrix multiplication and reshaping
        K_att = tf.matmul(QK_interaction_reshape, self.Wk)
        K_att = tf.transpose(K_att, perm=(0, 2, 1)) / (self.size_per_head ** 0.5)

        if len(x) == 4:
            K_att = K_att - (1 - tf.expand_dims(K_mask, axis=1)) * 1e8

        # Final operations
        K_att = tf.nn.softmax(K_att)
        K_att = tf.repeat(tf.expand_dims(K_att, axis=3), repeats=self.size_per_head, axis=3)
        global_k = tf.reduce_sum(tf.multiply(K_att, QK_interaction), axis=2)
        global_k_repeat = tf.repeat(tf.expand_dims(global_k, axis=2), self.now_input_shape[0][1], axis=2)

        # Final interaction
        QKQ_interaction = tf.multiply(global_k_repeat, Q_seq)
        QKQ_interaction = tf.transpose(QKQ_interaction, perm=(0, 2, 1, 3))
        QKQ_interaction = tf.reshape(QKQ_interaction, (-1, self.now_input_shape[0][1], self.nb_head * self.size_per_head))
        QKQ_interaction = tf.matmul(QKQ_interaction, self.WP)
        QKQ_interaction = tf.reshape(QKQ_interaction, (-1, self.now_input_shape[0][1], self.nb_head, self.size_per_head))
        QKQ_interaction = tf.transpose(QKQ_interaction, perm=(0, 2, 1, 3))
        QKQ_interaction = QKQ_interaction + Q_seq
        QKQ_interaction = tf.transpose(QKQ_interaction, perm=(0, 2, 1, 3))
        QKQ_interaction = tf.reshape(QKQ_interaction, (-1, self.now_input_shape[0][1], self.nb_head * self.size_per_head))
        return QKQ_interaction
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

import os
import requests
from tempfile import TemporaryDirectory
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

def download_file(url, dest_path):
    """Download a file from a URL and save it locally."""
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Check for request errors
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)  # Create directories if needed
    with open(dest_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:  # Filter out keep-alive new chunks
                file.write(chunk)

# Initialize temporary directory and paths
tmpdir = TemporaryDirectory()
data_set_size = "small"
data_path = f"dataset/{data_set_size}"#tmpdir.name
recreate = True
print(data_path)
train_news_file = os.path.join('news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', 'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', 'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', 'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", 'nrms.yaml')

# Get MIND dataset URLs
#mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)
mind_url = "https://recodatasets.z20.web.core.windows.net/newsrec/"
#mind_dev_dataset = "MINDlarge_dev.zip"
#mind_train_dataset = "MINDlarge_train.zip"
mind_dev_dataset = f"MIND{data_set_size}_dev.zip"
mind_train_dataset = f"MIND{data_set_size}_train.zip"
train_news_dataset = os.path.join(data_path, 'train', mind_train_dataset)
valid_news_dataset = os.path.join(data_path, 'valid', mind_dev_dataset)
# Download train dataset if not already present
if not os.path.exists(mind_train_dataset):
    download_file(os.path.join(mind_url, mind_train_dataset), os.path.join(data_path, 'train', mind_train_dataset))

# Download validation dataset if not already present
if not os.path.exists(mind_dev_dataset):
    download_file(os.path.join(mind_url, mind_dev_dataset), os.path.join(data_path, 'valid', mind_dev_dataset))

import zipfile
with zipfile.ZipFile(os.path.join(data_path, 'train', mind_train_dataset), 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

dataset/small


In [12]:

# Download utility files if not already present
#if not os.path.exists(yaml_file):
#    for util_file in mind_utils:
#        download_file(os.path.join('https://recodatasets.z20.web.core.windows.net/newsrec/', util_file),
#                      os.path.join(data_path, 'utils', util_file))

import os
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

def process_data():
    # Paths to save processed data
    processed_news_path = f"{data_path}/processed_news.pkl"
    processed_behaviors_path = f"{data_path}/processed_behaviors.pkl"
    
    # Check if processed data files exist
    if os.path.exists(processed_news_path) and os.path.exists(processed_behaviors_path) and not recreate:
        # Load processed data
        print("Loading processed data...")
        with open(processed_news_path, 'rb') as f:
            news_df = pickle.load(f)
        with open(processed_behaviors_path, 'rb') as f:
            behaviors_df = pickle.load(f)
        print("Processed data loaded.")
    else:
        # Load raw data
        print("Loading raw data...")
        behaviors_df = pd.read_csv(
            'behaviors.tsv',
            sep='\t',
            header=None,
            names=['UserID', 'Time', 'ClickedNews', 'Impressions'],
            dtype=str
        )
        news_df = pd.read_csv(
            'news.tsv',
            sep='\t',
            header=None,
            names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Entities', 'Keywords'],
            dtype=str
        )
        print("Raw data loaded.")
    
        # Process news_df
        print("Processing news data...")
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
        # Replace NaN values in 'Title' and 'Abstract' with empty strings
        news_df['Title'] = news_df['Title'].fillna('')
        news_df['Abstract'] = news_df['Abstract'].fillna('')
    
        # Tokenize 'Title' and 'Abstract' columns
        news_df['TitleTokens'] = news_df['Title'].apply(
            lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=32))
        news_df['AbstractTokens'] = news_df['Abstract'].apply(
            lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128))
        print("Tokenization complete.")
    
        # Encode categories and subcategories
        print("Encoding categories and subcategories...")
        category_encoder = LabelEncoder()
        sub_category_encoder = LabelEncoder()
        news_df['CategoryID'] = category_encoder.fit_transform(news_df['Category'])
        news_df['SubCategoryID'] = sub_category_encoder.fit_transform(news_df['SubCategory'])
        print("Encoding complete.")
    
        # Process behaviors_df
        print("Processing behaviors data...")
        behaviors_df['ClickedNews'] = behaviors_df['ClickedNews'].fillna('')
        behaviors_df['Impressions'] = behaviors_df['Impressions'].fillna('')
    
        # Process user behaviors
        behaviors_df['ClickedNewsList'] = behaviors_df['ClickedNews'].apply(
            lambda x: x.split(' ') if x else [])
        behaviors_df['ImpressionsList'] = behaviors_df['Impressions'].apply(
            lambda x: [imp.split('-') for imp in x.split(' ')] if x else [])
        print("Behaviors data processed.")
    
        # Save processed data
        print("Saving processed data...")
        with open(processed_news_path, 'wb') as f:
            pickle.dump(news_df, f)
        with open(processed_behaviors_path, 'wb') as f:
            pickle.dump(behaviors_df, f)
        print("Processed data saved.")
    
    print("Data is ready for analysis.")
    return behaviors_df, news_df 
behaviors_df, news_df  = process_data()

def process_data_old():
    # Load data
    behaviors_df = pd.read_csv('behaviors.tsv', sep='\t', header=None, names=['UserID', 'Time', 'ClickedNews', 'Impressions'])
    news_df = pd.read_csv('news.tsv', sep='\t', header=None, names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Entities', 'Keywords'])
    print("data loaded")
    print("tokenizer start")
    # Tokenize text data
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    news_df['TitleTokens'] = news_df['Title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=32))
    print("tokenizer end")
    # Replace NaN values in the 'Abstract' column with an empty string
    news_df['Abstract'] = news_df['Abstract'].fillna('')
    # Tokenize the 'Abstract' column
    
    print("tokenizer start")
    news_df['AbstractTokens'] = news_df['Abstract'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128))
    
    print("tokenizer end")
    print("replaced nan")
    # Encode categories and subcategories
    category_encoder = LabelEncoder()
    sub_category_encoder = LabelEncoder()
    
    news_df['CategoryID'] = category_encoder.fit_transform(news_df['Category'])
    news_df['SubCategoryID'] = sub_category_encoder.fit_transform(news_df['SubCategory'])
    print("news df fit transformed")
    # Map news ID to the processed news information
    news_dict = {row['NewsID']: row for _, row in news_df.iterrows()}
    
    # Replace NaN values in 'ClickedNews' and 'Impressions' columns with empty strings
    behaviors_df['ClickedNews'] = behaviors_df['ClickedNews'].fillna('')
    behaviors_df['Impressions'] = behaviors_df['Impressions'].fillna('')
    print("behaviours fillna")
    # Process user behaviors
    def process_behaviors(row):
        #print(row['ClickedNews'])
        clicked_news = row['ClickedNews'].split(' ')
        impressions = [imp.split('-') for imp in row['Impressions'].split(' ')]
        return clicked_news, impressions
    print("process behaviours start")
    #behaviors_df['ClickedNewsList'], behaviors_df['ImpressionsList'] = zip(*behaviors_df.apply(process_behaviors, axis=1))
    clicked_news_list = []
    impressions_list = []
    
    for index, row in behaviors_df.iterrows():
        clicked_news, impressions = process_behaviors(row)
        clicked_news_list.append(clicked_news)
        impressions_list.append(impressions)
    
    behaviors_df['ClickedNewsList'] = clicked_news_list
    behaviors_df['ImpressionsList'] = impressions_list
    print("process behaviours end")



Loading raw data...
Raw data loaded.
Processing news data...


/home/t/tf/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenization complete.
Encoding categories and subcategories...
Encoding complete.
Processing behaviors data...
Behaviors data processed.
Saving processed data...
Processed data saved.
Data is ready for analysis.


In [13]:
from torch.utils.data import Dataset, DataLoader
class NewsDataset(Dataset):
    def __init__(self, behaviors, news_dict):
        self.behaviors = behaviors
        self.news_dict = news_dict

    def __len__(self):
        return len(self.behaviors)

    def __getitem__(self, idx):
        clicked_news = [self.news_dict[nid]['TitleTokens'] for nid in self.behaviors.iloc[idx]['ClickedNewsList']]
        candidate_news = [self.news_dict[nid]['TitleTokens'] for nid, _ in self.behaviors.iloc[idx]['ImpressionsList']]
        labels = [int(label) for _, label in self.behaviors.iloc[idx]['ImpressionsList']]
        return torch.tensor(clicked_news), torch.tensor(candidate_news), torch.tensor(labels)

# Next step, training models

In [14]:
import dask.dataframe as dd
import pandas as pd
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from keras.layers import Input, Embedding, Dropout, LayerNormalization, add, Flatten, Dense, Activation, Dot, Lambda
from keras.models import Model
from keras.optimizers import Adam
import tensorflow.keras.backend as K
from datetime import datetime
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_HISTORY_LENGTH = 10
MAX_SENT_LENGTH = 32
@tf.function
def predict_user_model(model, clicked_news):
    return model(clicked_news, training=False)  # Ensure training=False for inference
from tensorflow.keras.layers import Input, Embedding, Dropout, LayerNormalization, Dense, Activation, Add, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Input, Embedding, Dropout, LayerNormalization, Dense, Activation, Add, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention

def build_model():
    word_dict_size = len(word_dict)
    embedding_dim = 256
    input_dim = 320  # Flattened input: 10 * 32
    target_length = 32
    
    # Define input layers
    encoder_input = Input(shape=(input_dim,), dtype='int32', name='encoder_input')
    decoder_input = Input(shape=(target_length,), dtype='int32', name='decoder_input')
    
    # Embedding layers
    encoder_emb = Embedding(input_dim=word_dict_size, output_dim=embedding_dim, mask_zero=True)(encoder_input)
    #encoder_emb = Embedding(input_dim=word_dict_size, output_dim=embedding_dim, input_length=input_dim, mask_zero=True)(encoder_input)
    encoder_emb = Dropout(0.2)(encoder_emb)
    
    decoder_emb = Embedding(input_dim=word_dict_size, output_dim=embedding_dim, mask_zero=True)(decoder_input)
    #decoder_emb = Embedding(input_dim=word_dict_size, output_dim=embedding_dim, input_length=target_length, mask_zero=True)(decoder_input)
    decoder_emb = Dropout(0.2)(decoder_emb)
    
    # Reshape embeddings to (batch_size, sequence_length, embedding_dim)
    encoder_emb = Reshape((input_dim, embedding_dim))(encoder_emb)
    decoder_emb = Reshape((target_length, embedding_dim))(decoder_emb)
    
    # Encoder self-attention
    encoder_output = MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(encoder_emb, encoder_emb)
    encoder_output = Dropout(0.2)(encoder_output)
    encoder_output = LayerNormalization()(encoder_output + encoder_emb)  # Residual connection
    
    # Decoder self-attention
    decoder_self_attention = MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(decoder_emb, decoder_emb)
    decoder_self_attention = Dropout(0.2)(decoder_self_attention)
    decoder_self_attention = LayerNormalization()(decoder_self_attention + decoder_emb)
    
    # Cross-attention
    decoder_cross_attention = MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(
        decoder_self_attention, encoder_output, encoder_output)
    decoder_cross_attention = Dropout(0.2)(decoder_cross_attention)
    decoder_cross_attention = LayerNormalization()(decoder_cross_attention + decoder_self_attention)
    
    # Feed-forward network
    ff_output = Dense(embedding_dim, activation='relu')(decoder_cross_attention)
    ff_output = Dropout(0.2)(ff_output)
    ff_output = LayerNormalization()(ff_output + decoder_cross_attention)
    
    # Output layer
    output = TimeDistributed(Dense(word_dict_size, activation='softmax'))(ff_output)
    
    # Define the model
    model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    return model




def build_model22():
    word_dict_size = len(word_dict)
    num_classes = len(tokenizer.vocab)  # Assuming you want to predict token IDs
    input_dim = MAX_HISTORY_LENGTH * MAX_SENT_LENGTH  # 10 * 32 = 320

    # Define input layers
    text_input = Input(shape=(MAX_HISTORY_LENGTH, MAX_SENT_LENGTH), dtype='int32', name='text_input')
    target_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32', name='target_input')  # For teacher forcing

    # Create mask if necessary (optional)
    # qmask = Lambda(lambda x: tf.cast(tf.cast(x, tf.bool), tf.float32))(text_input)

    # Embedding layer
    word_emb = Embedding(word_dict_size, 256, trainable=True)(text_input)
    word_emb = Dropout(0.2)(word_emb)

    # Fastformer layers
    hidden_word_emb = Fastformer(16, 16)([word_emb, word_emb])
    hidden_word_emb = Dropout(0.2)(hidden_word_emb)
    hidden_word_emb = LayerNormalization()(add([word_emb, hidden_word_emb]))

    hidden_word_emb_layer2 = Fastformer(16, 16)([hidden_word_emb, hidden_word_emb])
    hidden_word_emb_layer2 = Dropout(0.2)(hidden_word_emb_layer2)
    hidden_word_emb_layer2 = LayerNormalization()(add([hidden_word_emb, hidden_word_emb_layer2]))

    # Decoder part for sequence prediction
    decoder_emb = Embedding(word_dict_size, 256, trainable=True)(target_input)
    decoder_emb = Dropout(0.2)(decoder_emb)

    # Combine encoder and decoder
    combined = add([hidden_word_emb_layer2, decoder_emb])

    # Fastformer layer in decoder
    decoder_hidden = Fastformer(16, 16)([combined, combined])
    decoder_hidden = Dropout(0.2)(decoder_hidden)
    decoder_hidden = LayerNormalization()(add([combined, decoder_hidden]))

    # Output layer
    output = TimeDistributed(Dense(word_dict_size, activation='softmax'))(decoder_hidden)

    # Define the model
    model = Model([text_input, target_input], [output])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    return model



def build_model2():
    
    input_dim = MAX_HISTORY_LENGTH * MAX_SENT_LENGTH  # 10 * 32 = 320
    text_input = Input(shape=(input_dim,), dtype='int32')
    projection_layer = Dense(256, activation='relu')(text_input)
    # Fastformer originally had this MAX_SENT_LENGTH (256) text_input
    #text_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32') 
    qmask = Lambda(lambda x: tf.cast(tf.cast(x, tf.bool), tf.float32), 
                   output_shape=lambda s: s)(text_input)
    word_emb = Embedding(word_dict_size, 256, trainable=True)(text_input)

    word_emb = Dropout(0.2)(word_emb)
    hidden_word_emb = Fastformer(16, 16)([word_emb, word_emb, qmask, qmask])
    hidden_word_emb = Dropout(0.2)(hidden_word_emb)
    hidden_word_emb = LayerNormalization()(add([word_emb, hidden_word_emb]))

    hidden_word_emb_layer2 = Fastformer(16, 16)([hidden_word_emb, hidden_word_emb, qmask, qmask])
    hidden_word_emb_layer2 = Dropout(0.2)(hidden_word_emb_layer2)
    hidden_word_emb_layer2 = LayerNormalization()(add([hidden_word_emb, hidden_word_emb_layer2]))

    word_att = Flatten()(Dense(1)(hidden_word_emb_layer2))
    word_att = Activation('softmax')(word_att)
    text_emb = Dot((1, 1))([hidden_word_emb_layer2, word_att])
    classifier = Dense(4, activation='softmax')(text_emb)

    model = Model([text_input], [classifier])
    model.compile(loss=['categorical_crossentropy'], 
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])
    return model

# Parameters for the model
MAX_SENT_LENGTH = 256  # adjust based on your dataset
MAX_SENT_LENGTH = 32  # adjust based on your dataset
word_dict_size = len(word_dict)  # Vocabulary size from tokenizer or word_dict
batch_size = 64
epochs = 1
print(f"number of users:{len(behaviors_df['UserID'].unique())}")
save_dir = f"{data_path}/user_models"
os.makedirs(save_dir, exist_ok=True)

def generate_input_target_pairs(clicked_news_list, window_size=3):
    input_sequences = []
    targets = []
    
    # Create sliding window input-target pairs
    for i in range(len(clicked_news_list) - window_size):
        input_seq = clicked_news_list[i:i + window_size]  # Input: a sequence of articles
        target = clicked_news_list[i + window_size]  # Target: the next clicked article
        input_sequences.append(input_seq)
        targets.append(target)
    
    return input_sequences, targets

clicked_news_global = []
targets_global = []

print(f"len(behaviors_df):{len(behaviors_df)}")


number of users:50000
len(behaviors_df):156965


In [ ]:
import os
import gc
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import load_model  # Assuming you have a model saved
import pyarrow.parquet as pq
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def process_data_and_train():
    # Parameters
    CHUNK_SIZE = 400000  # Adjust based on your system's memory
    INTERMEDIATE_DIR = f"{data_path}/processed_chunks_{CHUNK_SIZE}"
    MODEL_SAVE_DIR = f"{data_path}/model_checkpoints"
    MAX_HISTORY_LENGTH = 10
    MAX_SENT_LENGTH = 32
    batch_size = 128
    epochs = 5
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    num_tokens = len(tokenizer.vocab)
    
    # Ensure necessary directories exist
    os.makedirs(INTERMEDIATE_DIR, exist_ok=True)
    os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
    
    # Load and preprocess news data
    print("Loading news data...")
    news_df = pd.read_csv('news.tsv', sep='\t', header=None,
                          names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Entities', 'Keywords'],
                          dtype=str)
    print("News data loaded.")
    
    # Fill missing values
    news_df['Title'] = news_df['Title'].fillna('')
    news_df['Abstract'] = news_df['Abstract'].fillna('')
    news_df['Category'] = news_df['Category'].fillna('UNKNOWN')  # Fill missing categories
    
    # Tokenize text data
    print("Tokenizing news titles and abstracts...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Tokenize 'Title' and 'Abstract' columns
    news_df['TitleTokens'] = news_df['Title'].apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=MAX_SENT_LENGTH))
    news_df['AbstractTokens'] = news_df['Abstract'].apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128))
    print("Tokenization complete.")
    
    # Encode categories and subcategories
    print("Encoding categories and subcategories...")
    category_encoder = LabelEncoder()
    sub_category_encoder = LabelEncoder()
    news_df['CategoryID'] = category_encoder.fit_transform(news_df['Category'])
    news_df['SubCategoryID'] = sub_category_encoder.fit_transform(news_df['SubCategory'])
    print("Encoding complete.")
    
    # Map news ID to the processed news information (including CategoryID)
    print("Creating news dictionary...")
    news_dict = news_df.set_index('NewsID')[['TitleTokens', 'CategoryID']].to_dict('index')
    print("News dictionary created.")
    
    # Function to generate input-target pairs
    def generate_input_target_pairs(clicked_news_list):
        input_sequences = []
        target_articles = []
        for i in range(1, len(clicked_news_list)):
            input_seq = clicked_news_list[:i]
            target_article = clicked_news_list[i]
            input_sequences.append(input_seq)
            target_articles.append(target_article)
        return input_sequences, target_articles
    
    # Process behaviors in chunks
    print("Processing behaviors in chunks...")
    behaviors_iter = pd.read_csv(
        'behaviors.tsv',
        sep='\t',
        header=None,
        names=['UserID', 'Time', 'ClickedNews', 'Impressions'],
        chunksize=CHUNK_SIZE,
        dtype=str
    )
    
    # Keep track of processed chunks
    processed_chunks = []
    
    for i, behaviors_chunk in enumerate(behaviors_iter):
        chunk_filename = os.path.join(INTERMEDIATE_DIR, f'processed_chunk_{i}.parquet')
    
        # Check if the chunk has already been processed
        if os.path.exists(chunk_filename):
            print(f"Chunk {i + 1} already processed. Skipping...")
            processed_chunks.append(chunk_filename)
            continue
    
        print(f"Processing chunk {i + 1}")
    
        # Replace NaN values in 'ClickedNews' and 'Impressions' columns with empty strings
        behaviors_chunk['ClickedNews'] = behaviors_chunk['ClickedNews'].fillna('')
        behaviors_chunk['Impressions'] = behaviors_chunk['Impressions'].fillna('')
    
        # Process user behaviors
        behaviors_chunk['ClickedNewsList'] = behaviors_chunk['ClickedNews'].apply(lambda x: x.split(' ') if x else [])
    
        # Generate input-target pairs for each row in the chunk
        input_sequences = []
        target_articles = []
    
        for idx, row in behaviors_chunk.iterrows():
            clicked_news_list = row['ClickedNewsList']
    
            # Skip if the clicked news list is empty or has only one item
            if len(clicked_news_list) < 2:
                continue
    
            input_seqs, target_articles_list = generate_input_target_pairs(clicked_news_list)
    
            # Accumulate results
            input_sequences.extend(input_seqs)
            target_articles.extend(target_articles_list)
    
        # Save processed data for this chunk
        processed_chunk_df = pd.DataFrame({'input_sequences': input_sequences, 'targets': target_articles})
        processed_chunk_df.to_parquet(chunk_filename, index=False)
        processed_chunks.append(chunk_filename)
    
        # Clean up
        del behaviors_chunk
        del processed_chunk_df
        gc.collect()
    
    print("Behavior processing complete.")
    
    # Now, process the accumulated data for model training/testing
    print("Preparing data for model training/testing...")
    
    def data_generator(chunk_files, batch_size):
        for chunk_file in chunk_files:
            processed_chunk = pd.read_parquet(chunk_file)
    
            input_seqs = []
            decoder_inputs = []
            target_seqs = []
    
            for idx, row in processed_chunk.iterrows():
                seq = row['input_sequences']  # List of clicked news IDs
                target_nid = row['targets']   # Target news ID
    
                # Get tokenized titles for clicked news
                tokens_list = [news_dict.get(nid, {'TitleTokens': [0]})['TitleTokens'] for nid in seq]
    
                # Pad each title's tokens to MAX_SENT_LENGTH
                tokens_list_padded = pad_sequences(
                    tokens_list,
                    maxlen=MAX_SENT_LENGTH,
                    padding='post',
                    truncating='post',
                    dtype='int32',
                    value=0
                )
    
                # Pad or truncate the sequence of tokenized titles to MAX_HISTORY_LENGTH
                if len(tokens_list_padded) < MAX_HISTORY_LENGTH:
                    # Pad with zeros at the end
                    padding = np.zeros((MAX_HISTORY_LENGTH - len(tokens_list_padded), MAX_SENT_LENGTH), dtype='int32')
                    tokens_list_padded = np.vstack([tokens_list_padded, padding])
                else:
                    tokens_list_padded = tokens_list_padded[-MAX_HISTORY_LENGTH:]  # Use the most recent clicked news
    
                # Flatten the input sequence from (10, 32) to (320,)
                input_seq_flat = tokens_list_padded.flatten()  # Shape: (320,)
                input_seqs.append(input_seq_flat)
    
                # Get tokenized title for the target news
                target_tokens = news_dict.get(target_nid, {'TitleTokens': [0]})['TitleTokens']
                target_tokens_padded = pad_sequences(
                    [target_tokens],
                    maxlen=MAX_SENT_LENGTH,
                    padding='post',
                    truncating='post',
                    dtype='int32',
                    value=0
                )[0]
                target_seqs.append(target_tokens_padded)
    
                # Prepare decoder input by shifting the target sequence right and adding a start token
                start_token = 1  # Replace with your actual start token ID
                decoder_input_seq = np.insert(target_tokens_padded, 0, start_token)[:-1]
                decoder_inputs.append(decoder_input_seq)
    
            input_seqs = np.array(input_seqs)          # Shape: (num_samples, 320)
            decoder_inputs = np.array(decoder_inputs)  # Shape: (num_samples, 32)
            target_seqs = np.array(target_seqs)        # Shape: (num_samples, 32)
    
            # Convert target_seqs to integers and reshape for sparse categorical crossentropy
            target_seqs = np.expand_dims(target_seqs, -1)  # Shape: (num_samples, 32, 1)
    
            print(f"Chunk {chunk_file}: Encoder input shape: {input_seqs.shape}, Decoder input shape: {decoder_inputs.shape}, Target output shape: {target_seqs.shape}")
    
            num_samples = len(input_seqs)
            for start_idx in range(0, num_samples, batch_size):
                end_idx = min(start_idx + batch_size, num_samples)
                #yield [input_seqs[start_idx:end_idx], decoder_inputs[start_idx:end_idx]], target_seqs[start_idx:end_idx]
                yield (input_seqs[start_idx:end_idx], decoder_inputs[start_idx:end_idx]), target_seqs[start_idx:end_idx]

    
            # Clean up
            del processed_chunk
            gc.collect()
    
    # Build or load the model
    model_save_path = os.path.join(MODEL_SAVE_DIR, f"fastformer_model_MIND_L_{len(processed_chunks)}_chunks_of_{CHUNK_SIZE}.keras")
    if os.path.exists(model_save_path):
        print(f"Loading existing model from {model_save_path}")
        model = load_model(model_save_path, custom_objects={'Fastformer': Fastformer})
    else:
        print("No existing model found. Defining and compiling a new model.")
        model = build_model()
    import tensorflow as tf

    output_signature = (
        (
            tf.TensorSpec(shape=(None, 320), dtype=tf.int32),   # encoder_input
            tf.TensorSpec(shape=(None, 32), dtype=tf.int32)     # decoder_input
        ),
        tf.TensorSpec(shape=(None, 32, 1), dtype=tf.int32)      # target_output
    )
    print("Prepare data generator")
    # Prepare data generator for training
    #data_gen = data_generator(processed_chunks, batch_size)
    data_gen = lambda: data_generator(processed_chunks, batch_size)
    
    dataset = tf.data.Dataset.from_generator(
        data_gen,
        output_signature=output_signature
    ).repeat()
    
    # Optional: Prefetch for performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    print("Calculate total number of samples for training")
    total_samples = 0
    for chunk_file in processed_chunks:
        processed_chunk = pd.read_parquet(chunk_file)
        total_samples += len(processed_chunk)
        del processed_chunk
        gc.collect()
    
    steps_per_epoch = total_samples // batch_size + int(total_samples % batch_size != 0)
    print(f"Total samples: {total_samples}, Steps per epoch: {steps_per_epoch}")
    
    model_ready = True
    if not model_ready:
        # Train the model using the generator
        print("Starting model training...")
        from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
        
        checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, monitor='loss', mode='min')
        early_stop = EarlyStopping(monitor='loss', patience=3, mode='min')
        
        model.fit(
            x=dataset,
            epochs=epochs,
            verbose=1,
            steps_per_epoch=steps_per_epoch,
            callbacks=[checkpoint, early_stop]
        )
        
        # Save the model after training
        model.save(model_save_path)
        print(f"Model saved to {model_save_path}")
    
    # Evaluate the model
    print("Evaluating the model...")
    # Prepare data generator for evaluation
    testing_chunks = [processed_chunks[0]]
    # Prepare data generator for evaluation
    data_gen_eval = data_generator(testing_chunks, batch_size)
    
    # Collect predictions and true labels
    y_true = []
    y_pred = []
    
    for (input_batch, decoder_batch), target_batch in data_gen_eval:
        preds = model.predict([input_batch, decoder_batch])
        preds = np.argmax(preds, axis=-1)  # Shape: (batch_size, 32)
        y_pred.extend(preds.flatten())
        y_true.extend(target_batch.flatten())
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, f1_score
    
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    
    print(f"Evaluation results - Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

process_data_and_train()

/home/t/tf/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading news data...
News data loaded.
Tokenizing news titles and abstracts...


/home/t/tf/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenization complete.
Encoding categories and subcategories...
Encoding complete.
Creating news dictionary...
News dictionary created.
Processing behaviors in chunks...
Chunk 1 already processed. Skipping...
Behavior processing complete.
Preparing data for model training/testing...
Loading existing model from dataset/small/model_checkpoints/fastformer_model_MIND_L_1_chunks_of_400000.keras


I0000 00:00:1727539085.251085   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727539085.251206   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727539085.251252   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727539085.251265   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727539085.251275   59597 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Prepare data generator
Calculate total number of samples for training
Total samples: 4953912, Steps per epoch: 38703
Evaluating the model...
Chunk dataset/small/processed_chunks_400000/processed_chunk_0.parquet: Encoder input shape: (4953912, 320), Decoder input shape: (4953912, 32), Target output shape: (4953912, 32, 1)


I0000 00:00:1727540654.447356   61042 service.cc:146] XLA service 0x7f6208003360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727540654.454643   61042 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 SUPER, Compute Capability 8.9
I0000 00:00:1727540654.454675   61042 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2024-09-28 19:24:15.054891: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-28 19:24:16.105732: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-28 19:24:20.337047: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_20', 4 bytes spill stores, 4 bytes spill loads

2024-09-28 19:24:22.904987: I 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

I0000 00:00:1727540667.340007   61042 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 176ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms

In [ ]:
import tracemalloc
def process_data_and_train_old():
    tracemalloc.start()
    # Assume behaviors_df is already loaded in your environment
    # If not, load it using:
    # behaviors_df = pd.read_csv('behaviors.csv')  # Replace with your actual data loading method
    
    import pandas as pd
    import os
    import gc
    from transformers import BertTokenizer
    from sklearn.preprocessing import LabelEncoder
    
    # Ensure necessary directories exist
    INTERMEDIATE_DIR = 'processed_chunks'
    os.makedirs(INTERMEDIATE_DIR, exist_ok=True)
    
    # Load news data
    print("Loading news data...")
    news_df = pd.read_csv('news.tsv', sep='\t', header=None, names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Entities', 'Keywords'], dtype=str)
    print("News data loaded.")
    
    # Tokenize text data
    print("Tokenizing news titles and abstracts...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Replace NaN values in 'Title' and 'Abstract' with empty strings
    news_df['Title'] = news_df['Title'].fillna('')
    news_df['Abstract'] = news_df['Abstract'].fillna('')
    
    # Tokenize 'Title' and 'Abstract' columns
    news_df['TitleTokens'] = news_df['Title'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=32))
    news_df['AbstractTokens'] = news_df['Abstract'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128))
    print("Tokenization complete.")
    
    # Encode categories and subcategories
    print("Encoding categories and subcategories...")
    category_encoder = LabelEncoder()
    sub_category_encoder = LabelEncoder()
    news_df['CategoryID'] = category_encoder.fit_transform(news_df['Category'])
    news_df['SubCategoryID'] = sub_category_encoder.fit_transform(news_df['SubCategory'])
    print("Encoding complete.")
    
    # Map news ID to the processed news information
    print("Creating news dictionary...")
    news_dict = news_df.set_index('NewsID').T.to_dict('list')
    print("News dictionary created.")
    
    # Function to generate input-target pairs
    def generate_input_target_pairs(clicked_news_list):
        input_sequences = []
        target_articles = []
        for i in range(1, len(clicked_news_list)):
            input_seq = clicked_news_list[:i]
            target_article = clicked_news_list[i]
            input_sequences.append(input_seq)
            target_articles.append(target_article)
        return input_sequences, target_articles
    
    # Initialize global lists
    clicked_news_global = []
    targets_global = []
    
    # Set the chunk size based on available memory
    CHUNK_SIZE = 5000  # Adjust this value as needed
    
    # Read behaviors.tsv in chunks
    print("Processing behaviors in chunks...")
    behaviors_iter = pd.read_csv(
        'behaviors.tsv',
        sep='\t',
        header=None,
        names=['UserID', 'Time', 'ClickedNews', 'Impressions'],
        chunksize=CHUNK_SIZE,
        dtype=str
    )
    
    for i, behaviors_chunk in enumerate(behaviors_iter):
        print(f"Processing chunk {i + 1}")
    
        # Replace NaN values in 'ClickedNews' and 'Impressions' columns with empty strings
        behaviors_chunk['ClickedNews'] = behaviors_chunk['ClickedNews'].fillna('')
        behaviors_chunk['Impressions'] = behaviors_chunk['Impressions'].fillna('')
    
        # Process user behaviors
        behaviors_chunk['ClickedNewsList'] = behaviors_chunk['ClickedNews'].apply(lambda x: x.split(' ') if x else [])
        # Impressions processing is omitted if not needed
    
        # Generate input-target pairs for each row in the chunk
        for idx, row in behaviors_chunk.iterrows():
            clicked_news_list = row['ClickedNewsList']
    
            # Skip if the clicked news list is empty or has only one item
            if len(clicked_news_list) < 2:
                continue
    
            input_seqs, target_articles = generate_input_target_pairs(clicked_news_list)
    
            # Accumulate results
            clicked_news_global.extend(input_seqs)
            targets_global.extend(target_articles)
    
        # Clean up
        del behaviors_chunk
        gc.collect()
    
        # Optional: Monitor memory usage
        # current, peak = tracemalloc.get_traced_memory()
        # print(f"After chunk {i + 1}: Current memory usage is {current / 10**6:.2f}MB; Peak was {peak / 10**6:.2f}MB")
    
    print("Processing complete.")
    
    # At this point, clicked_news_global and targets_global contain the accumulated data
    print(f"Total number of input sequences: {len(clicked_news_global)}")
    print(f"Total number of target articles: {len(targets_global)}")
    
    # Optional: Save the results to disk if needed
    # pd.DataFrame({'input_sequences': clicked_news_global, 'targets': targets_global}).to_parquet('final_data.parquet')
    
    
    # Clean up intermediate files if desired
    # for file in processed_chunk_files:
    #     os.remove(file)
    
    print("Processing complete. 'clicked_news_global' and 'targets_global' are ready for use.")
    print(f"len(clicked_news_global)={len(clicked_news_global)}")
    print(f"len(targets_global)={len(targets_global)}")
    # Tokenize the input sequences and pad them
    clicked_news_padded = pad_sequences([[news_dict[nid]['TitleTokens'] for nid in seq] for seq in clicked_news_global],
                                        maxlen=MAX_SENT_LENGTH, padding='post', truncating='post')
    
    # Convert the target articles into embeddings (you may tokenize them similarly)
    target_news_padded = np.array([news_dict[nid]['TitleTokens'] for nid in targets_global])
    print(f"len(clicked_news_padded)={len(clicked_news_padded)}")
    print(f"len(targets_global)={len(targets_global)}")
    
    # Train/test split (80/20)
    split_idx = int(0.8 * len(clicked_news_padded))
    train_index = np.arange(split_idx)
    test_index = np.arange(split_idx, len(clicked_news_padded))
    
    # Convert target articles to one-hot encoding if necessary (for classification)
    # If predicting classes, use `to_categorical` to one-hot encode the target labels
    target_labels_onehot = to_categorical(target_news_padded, num_classes=4)
    
    print(f"len(target_labels_onehot)={len(target_labels_onehot)}")
    print(f"clicked_news_padded={clicked_news_padded}")
    print(f"target_labels_onehot={target_labels_onehot}")
    # Now fit the model
    model.fit(clicked_news_padded[train_index], 
              target_labels_onehot[train_index], 
              shuffle=True, 
              batch_size=batch_size, 
              epochs=epochs, 
              verbose=1)
    model_save_path = os.path.join(save_dir, f"global_fastformer_model.h5")
    model.save(model_save_path)
    
    if True:
        print(f"Evaluating model: {model_save_path}")
        y_pred = predict_user_model(model, clicked_news_padded[test_index])
        #y_pred = model.predict(clicked_news[test_index], batch_size=128, verbose=1)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = labels[test_index]
        
        # Calculate metrics
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        
        print(f" model: {model_save_path} - Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import os

# Define your generate_input_target_pairs function
def generate_input_target_pairs(clicked_news_list):
    # Placeholder function - replace with your actual implementation
    input_sequences = []
    target_articles = []
    for i in range(1, len(clicked_news_list)):
        input_sequences.append(clicked_news_list[:i])
        target_articles.append(clicked_news_list[i])
    return input_sequences, target_articles

# Optimized process_chunk function using generators
def process_chunk(df_chunk):
    def data_generator():
        for _, row in df_chunk.iterrows():
            clicked_news_list = row['ClickedNewsList']
            input_seqs, target_articles = generate_input_target_pairs(clicked_news_list)
            for seq, target in zip(input_seqs, target_articles):
                yield {'input_sequences': seq, 'targets': target}
    # Create DataFrame from generator
    return pd.DataFrame(data_generator())

# Set the chunk size based on available memory
CHUNK_SIZE = 100000  # Adjust this value as needed

# Path to save intermediate processed chunks
INTERMEDIATE_DIR = 'processed_chunks'
os.makedirs(INTERMEDIATE_DIR, exist_ok=True)

# If behaviors_df is already loaded in memory and is a DataFrame
behaviors_df = pd.read_csv('behaviors.csv')  # Replace with your actual data loading method

num_rows = len(behaviors_df)
num_chunks = num_rows // CHUNK_SIZE + int(num_rows % CHUNK_SIZE != 0)

processed_chunk_files = []

for i in range(num_chunks):
    start_idx = i * CHUNK_SIZE
    end_idx = min((i + 1) * CHUNK_SIZE, num_rows)
    df_chunk = behaviors_df.iloc[start_idx:end_idx]

    print(f"Processing chunk {i + 1}/{num_chunks} (rows {start_idx} to {end_idx - 1})")

    # Process the chunk
    processed_chunk = process_chunk(df_chunk)

    # Save the processed chunk to disk
    chunk_filename = os.path.join(INTERMEDIATE_DIR, f'processed_chunk_{i}.parquet')
    processed_chunk.to_parquet(chunk_filename, index=False)
    processed_chunk_files.append(chunk_filename)

    # Optional: delete variables to free memory
    del df_chunk
    del processed_chunk

# Combine all processed chunks into a final DataFrame
print("Combining all processed chunks into the final DataFrame...")
processed_chunks = [pd.read_parquet(file) for file in processed_chunk_files]
final_df = pd.concat(processed_chunks, ignore_index=True)

# Clean up intermediate files if desired
# for file in processed_chunk_files:
#     os.remove(file)

print("Processing complete. Final DataFrame is ready for use.")

In [ ]:
keras.backend.clear_session() 
import tensorflow as tf
text_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
#qmask=Lambda(lambda x:  K.cast(K.cast(x,'bool'),'float32'))(text_input)
#qmask = Lambda(lambda x: K.cast(K.cast(x, 'bool'), 'float32'), 
#               output_shape=lambda s: s)(text_input)
qmask = Lambda(lambda x: tf.cast(tf.cast(x, tf.bool), tf.float32), 
               output_shape=lambda s: s)(text_input)
word_emb = Embedding(len(word_dict),256, trainable=True)(text_input)

#pos_emb = Embedding(MAX_SENT_LENGTH, 256, trainable=True)(Lambda(lambda x:K.zeros_like(x,dtype='int32')+K.arange(x.shape[1]))(text_input))
#word_emb  =add([word_emb ,pos_emb])
#We find that position embedding is not important on this dataset and we removed it for simplicity. If needed, please uncomment the two lines above

word_emb=Dropout(0.2)(word_emb)

hidden_word_emb = Fastformer(16,16)([word_emb,word_emb,qmask,qmask])
hidden_word_emb = Dropout(0.2)(hidden_word_emb)
hidden_word_emb = LayerNormalization()(add([word_emb,hidden_word_emb])) 
#if there is no layer norm in old version, please import an external layernorm class from a higher version.

hidden_word_emb_layer2 = Fastformer(16,16)([hidden_word_emb,hidden_word_emb,qmask,qmask])
hidden_word_emb_layer2 = Dropout(0.2)(hidden_word_emb_layer2)
hidden_word_emb_layer2 = LayerNormalization()(add([hidden_word_emb,hidden_word_emb_layer2]))

#without FFNN for simplicity

word_att = Flatten()(Dense(1)(hidden_word_emb_layer2))
word_att = Activation('softmax')(word_att)
text_emb = Dot((1, 1))([hidden_word_emb_layer2 , word_att])
classifier = Dense(4, activation='softmax')(text_emb)
                                      
model = Model([text_input], [classifier])
#model.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=0.001), metrics=['acc'])
model.compile(loss=['categorical_crossentropy'], 
              optimizer=Adam(learning_rate=0.001),
              metrics=['acc'])

for i in range(1):
    model.fit(news_words[train_index],to_categorical(label)[train_index],shuffle=True,batch_size=64, epochs=1,verbose=1)


    y_pred = model.predict([news_words[test_index] ], batch_size=128, verbose=1)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = label[test_index]
    acc = accuracy_score(y_true, y_pred)
    report = f1_score(y_true, y_pred, average='macro')  
    print(acc)
    print(report)


In [ ]:
print(to_categorical(label, num_classes=4)[11])
print(to_categorical(labels, num_classes=4)[11])

In [ ]:
print(len(labels))
print(len(clicked_news))
print(labels)
print(clicked_news)
print(clicked_news_padded)
user_id = "U532401"
user_behaviors = behaviors_df[behaviors_df['UserID'] == user_id]
print(user_behaviors['ClickedNewsList'].explode())
# Extract impressions for this user, which includes both the articles shown and the labels
impressions = user_behaviors['ImpressionsList'].explode()

# Extract the news articles and the corresponding labels from the impressions
clicked_news2 = [news_dict[nid]['TitleTokens'] for nid, label in impressions]
labels2 = [int(label) for nid, label in impressions]
print(impressions)
print(clicked_news2)
print(labels2)
print(len(clicked_news2))
print(len(labels2))

In [ ]:
def make_model_for_each_user:
    # Loop through each user and train a model based on their specific data
    for user_id in behaviors_df['UserID'].unique():
        keras.backend.clear_session()  # Clear previous model to start fresh for each user
        
        # Filter the dataset for the current user
        user_behaviors = behaviors_df[behaviors_df['UserID'] == user_id]
        """
        # Extract the news interactions for this user
        clicked_news = [news_dict[nid]['TitleTokens'] for nid in user_behaviors['ClickedNewsList'].explode()]
        labels = [int(label) for nid, label in user_behaviors['ImpressionsList'].explode()]
    
        # Convert clicked news and labels to numpy arrays
        clicked_news = np.array(clicked_news)
        labels = np.array(labels)
        """
        import numpy as np
        from tensorflow.keras.preprocessing.sequence import pad_sequences
        
        # Assume MAX_SENT_LENGTH is defined as 256
        MAX_SENT_LENGTH = 256
        
        # Extract the news interactions for this user and pad sequences to the same length
        #clicked_news = [news_dict[nid]['TitleTokens'] for nid in user_behaviors['ClickedNewsList'].explode()]
        
        # Pad sequences to ensure they have the same length (MAX_SENT_LENGTH)
        #clicked_news_padded = pad_sequences(clicked_news, maxlen=MAX_SENT_LENGTH, padding='post', truncating='post')
        
        # Extract labels for this user
        #labels = [int(label) for nid, label in user_behaviors['ImpressionsList'].explode()]
    
        impressions = user_behaviors['ImpressionsList'].explode()
        
        # Extract the news articles and the corresponding labels from the impressions
        clicked_news = [news_dict[nid]['TitleTokens'] for nid, label in impressions]
        labels = [int(label) for nid, label in impressions]
        clicked_news_padded = pad_sequences(clicked_news, maxlen=MAX_SENT_LENGTH, padding='post', truncating='post')
        # Convert clicked news and labels to numpy arrays
        clicked_news_padded = np.array(clicked_news_padded)
        labels = np.array(labels)
        clicked_news = clicked_news_padded
        # Train/test split (you can adjust this as needed)
        split_idx = int(0.8 * len(clicked_news))
        import random
        #index=np.arange(len(label))
        #train_index=index[:120000]
        #test_index=index[120000:]
        train_index = np.arange(split_idx)
        test_index = np.arange(split_idx, len(clicked_news))
    
        # Build the model (same model as in the original code)
        model = build_model()
    
        # Train the model using the user-specific data
        print(f"Training model for user {user_id}. Click news:{len(clicked_news)}")
        model.fit(clicked_news[train_index], 
                  to_categorical(labels, num_classes=4)[train_index], 
                  shuffle=True, 
                  batch_size=batch_size, 
                  epochs=epochs, 
                  verbose=1)
        model_save_path = os.path.join(save_dir, f"user_{user_id}_model.h5")
        model.save(model_save_path)
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Model for user {user_id} saved to {model_save_path}")
        # Predict and evaluate on the test data
        if False:
            print(f"Evaluating model for user {user_id}...")
            y_pred = predict_user_model(model, clicked_news_padded[test_index])
            #y_pred = model.predict(clicked_news[test_index], batch_size=128, verbose=1)
            y_pred = np.argmax(y_pred, axis=1)
            y_true = labels[test_index]
            
            # Calculate metrics
            acc = accuracy_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred, average='macro')
            
            print(f"User {user_id} - Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")